# 🚀 FinSentiment 3.0: Sector Screener (Batch Edition)

Versión escalable para analizar múltiples empresas del sector tecnológico de forma secuencial y comparativa.

### Nuevas Características (Fase 7):
1. **Batch Processing**: Análisis automático de múltiples tickers definidos por el usuario.
2. **Robustez**: Manejo de errores encapsulado (si falla uno, sigue el siguiente).
3. **Screener Visual**: Ranking de sentimiento (Z-Score) para comparar empresas.
4. **Core v2.1**: Filtros legales y normalización estadística ya integrados.

In [ ]:
# @title 1. Instalación de Dependencias
!pip install sec-edgar-downloader transformers torch pandas numpy matplotlib seaborn beautifulsoup4 yfinance

In [ ]:
# @title 2. Definición del Motor (Core Engine v2.1)

import os
import re
import glob
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf
from collections import Counter
from datetime import datetime, timedelta
from bs4 import BeautifulSoup
from sec_edgar_downloader import Downloader
from transformers import BertTokenizer, BertForSequenceClassification

# Configurar estilos
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = [14, 7]

# --- MÓDULO 1: INGESTA ---
class SECLoader:
    def __init__(self, data_dir="data", email="research@example.com", company="Personal Research"):
        self.data_dir = data_dir
        os.makedirs(os.path.join(data_dir, "raw"), exist_ok=True)
        os.makedirs(os.path.join(data_dir, "processed"), exist_ok=True)
        self.downloader = Downloader(company, email, os.path.join(data_dir, "raw"))

    def download_filings(self, ticker: str, amount: int = 1):
        path = os.path.join(self.data_dir, "raw", "sec-edgar-filings", ticker)
        # En batch mode, a veces conviene forzar descarga si amount es bajo
        print(f"[>] ({ticker}) Iniciando descarga de {amount} reporte(s)...")
        try:
            self.downloader.get("10-K", ticker, limit=amount)
        except Exception as e:
            print(f"[!] Error en descarga {ticker}: {e}")

    def extract_date(self, content: str) -> str:
        patterns = [
            r'FILED AS OF DATE:\s+(\d{8})',
            r'CONFORMED PERIOD OF REPORT:\s+(\d{8})'
        ]
        for p in patterns:
            match = re.search(p, content)
            if match:
                date_str = match.group(1)
                return f"{date_str[:4]}-{date_str[4:6]}-{date_str[6:]}"
        return None

    def extract_mda(self, html_content: str) -> str:
        soup = BeautifulSoup(html_content, 'html.parser')
        text = soup.get_text(separator='\n')
        patterns = [r'Item\s+7\.\s+Management', r"Management's\s+Discussion", r'Item\s+7\.']
        start_idx = -1
        for p in patterns:
            match = re.search(p, text, re.IGNORECASE)
            if match: start_idx = match.start(); break
        if start_idx == -1: return text[:50000]
        return text[start_idx:start_idx+30000]

    def process_filings(self, ticker: str):
        raw_path = os.path.join(self.data_dir, "raw", "sec-edgar-filings", ticker)
        processed_data = []
        for root, _, files in os.walk(raw_path):
            for file in files:
                if file.lower().endswith(".txt") and "primary" not in file:
                    try:
                        with open(os.path.join(root, file), 'r', encoding='utf-8', errors='ignore') as f:
                            content = f.read()
                        mda = self.extract_mda(content)
                        date = self.extract_date(content)
                        if len(mda) > 500:
                            processed_data.append({'text': mda, 'date': date, 'accession': file})
                    except: pass
        # Ordenar por fecha reciente
        return sorted(processed_data, key=lambda x: x.get('date', '1900'), reverse=True)

# --- MÓDULO 2 NOISE FILTER ---
class TextPreprocessor:
    def clean_text(self, text):
        text = re.sub(r'\s+', ' ', text)
        return "".join(ch for ch in text if ch.isprintable()).strip()
    
    def is_legal_noise(self, sentence: str) -> bool:
        legal_keywords = [
            'forward-looking', 'safe harbor', 'uncertainty', 'may differ', 
            'subject to error', 'actual results', 'factors that could cause',
            'statements regarding', 'cautionary note', 'risk factors', 
            'include but are not limited to', 'assumptions'
        ]
        return any(kw in sentence.lower() for kw in legal_keywords)

    def split_sentences(self, text):
        sentences = re.split(r'(?<=[.!?])\s+(?=[A-Z])', text)
        valid_sentences = []
        for s in sentences:
            s = s.strip()
            if len(s) > 20 and len(s.split()) >= 4:
                if not self.is_legal_noise(s):
                    valid_sentences.append(s)
        return valid_sentences

# --- MÓDULO 3 FINBERT ---
class FinBertModel:
    def __init__(self):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        print(f"[*] Cargando FinBERT en {self.device}...")
        self.tokenizer = BertTokenizer.from_pretrained("ProsusAI/finbert")
        self.model = BertForSequenceClassification.from_pretrained("ProsusAI/finbert").to(self.device)
        self.labels = {0: 'positive', 1: 'negative', 2: 'neutral'}

    def predict(self, sentences):
        if not sentences: return pd.DataFrame()
        batch_size = 32
        results = []
        for i in range(0, len(sentences), batch_size):
            batch = sentences[i:i+batch_size]
            inputs = self.tokenizer(batch, return_tensors="pt", padding=True, truncation=True, max_length=512).to(self.device)
            with torch.no_grad():
                outputs = self.model(**inputs)
                probs = torch.nn.functional.softmax(outputs.logits, dim=-1).cpu().numpy()
            for j, s in enumerate(batch):
                results.append({
                    "sentence": s,
                    "pos_val": probs[j][0],
                    "neg_val": probs[j][1]
                })
        return pd.DataFrame(results)


In [ ]:
# @title 3. Ejecución Batch con Selector de Tickers

# Inicializar Modelos
print("[*] Inicializando Motor NLP...")
loader = SECLoader()
model = FinBertModel()
prep = TextPreprocessor()

# --- CONFIGURACIÓN DE USUARIO ---
TICKERS_INPUT = "META, AAPL, MSFT, GOOGL, AMZN" # @param {type:"string"}
NUM_REPORTS = 1 # @param {type:"integer"}

# Parsear tickers (separados por coma)
TICKERS = [t.strip().upper() for t in TICKERS_INPUT.split(',') if t.strip()]
print(f"\n📝 TARGETS: {TICKERS}")
print(f"📑 REPORTES POR EMPRESA: {NUM_REPORTS}")

def analyze_ticker(ticker_symbol, amount=1):
    """Función Maestra: Procesa una empresa de principio a fin."""
    print(f"\n" + "-"*50)
    print(f" 🏢 PROCESANDO AGENTE: {ticker_symbol}")
    print("-"*50)
    try:
        # 1. Download
        loader.download_filings(ticker_symbol, amount=amount)
        
        # 2. Process
        docs = loader.process_filings(ticker_symbol)
        if not docs:
            print(f"[!] No se encontraron reportes legibles para {ticker_symbol}")
            return None
            
        # Tomamos el/los reportes solicitados (hasta donde haya)
        reports_to_scan = docs[:amount]
        print(f"[>] Analizando {len(reports_to_scan)} documento(s)...")
        
        # Para el screener simple, usamos el MÁS RECIENTE, 
        # pero podriamos promediar si el usuario pide más.
        # Aquí tomaremos el último (índice 0 tras el sort) para el 'current sentiment'.
        latest_doc = reports_to_scan[0]
        report_date = latest_doc.get('date', 'Unknown')
        print(f"[>] Analizando reporte principal del {report_date}...")
        
        # 3. Clean & Predict
        sentences = prep.split_sentences(prep.clean_text(latest_doc['text']))
        print(f"[i] Oraciones extraídas: {len(sentences)}")
        
        df = model.predict(sentences)
        
        if df.empty:
            return None
            
        # 4. Calculate Metrics
        net_score = df['pos_val'].mean() - df['neg_val'].mean()
        print(f"[ok] Net Score Calculado: {net_score:.4f}")
        
        return {
            'ticker': ticker_symbol,
            'date': report_date,
            'net_score': net_score
        }
        
    except Exception as e:
        print(f"[ERROR] Fallo crítico en {ticker_symbol}: {e}")
        return None

# --- BUCLE PRINCIPAL ---
sector_results = []

for t in TICKERS:
    res = analyze_ticker(t, amount=NUM_REPORTS)
    if res:
        sector_results.append(res)

# Construir DataFrame
df_sector = pd.DataFrame(sector_results)

if not df_sector.empty:
    # Calcular Z-Score relativo al GRUPO (Sectorial)
    sector_mean = df_sector['net_score'].mean()
    sector_std = df_sector['net_score'].std()
    if sector_std == 0: sector_std = 1
    
    df_sector['z_score'] = (df_sector['net_score'] - sector_mean) / sector_std

    # --- MARKET DATA INTEGRATION (ALPHA HUNTER) ---
    print("\n[*] Descargando datos de mercado (6 meses)...")
    end_date = datetime.now()
    start_date = end_date - timedelta(days=180)
    
    # Descarga optimizada en batch
    tickers_list = df_sector['ticker'].tolist()
    try:
        market_data = yf.download(tickers_list, start=start_date, end=end_date)['Close']
        
        # Calcular Retorno 6M (%)
        if len(tickers_list) == 1:
            # pandas Series handling
            price_return = (market_data.iloc[-1] - market_data.iloc[0]) / market_data.iloc[0] * 100
            df_sector['price_return_6m'] = price_return
        else:
            # pandas DataFrame handling
            returns = (market_data.iloc[-1] - market_data.iloc[0]) / market_data.iloc[0] * 100
            # Mapear los retornos al dataframe (indice de returns es ticker)
            df_sector['price_return_6m'] = df_sector['ticker'].map(returns)
            
    except Exception as e:
        print(f"[!] Error calculando retornos: {e}")
        df_sector['price_return_6m'] = 0.0

    print("\n" + "="*40)
    print(" SECTOR SCREENER RESULTS (Alpha)")
    print("="*40)
    display(df_sector.sort_values('z_score', ascending=False))
    
    # --- VISUALIZACIÓN 1: BARRAS Z-SCORE ---
    plt.figure(figsize=(10, max(6, len(df_sector)*0.6)))
    plot_data = df_sector.sort_values('z_score', ascending=False)
    colors = ['#2ecc71' if x >= 0 else '#e74c3c' for x in plot_data['z_score']]
    sns.barplot(x='z_score', y='ticker', data=plot_data, palette=colors)
    plt.title('Sentiment Leaderboard (Z-Score)', fontsize=14, fontweight='bold')
    plt.xlabel('Z-Score')
    plt.show()
    
    # --- VISUALIZACIÓN 2: ALPHA SCATTER PLOT ---
    plt.figure(figsize=(12, 8))
    
    sns.scatterplot(
        data=df_sector, 
        x='z_score', 
        y='price_return_6m', 
        s=150, 
        color='#2E86AB',
        edgecolor='black',
        alpha=0.8
    )
    
    # Etiquetas (Labels)
    for i in range(df_sector.shape[0]):
        plt.text(
            x=df_sector.z_score.iloc[i]+0.02, 
            y=df_sector.price_return_6m.iloc[i]+0.5, 
            s=df_sector.ticker.iloc[i], 
            fontweight='bold',
            fontsize=10
        )

    # Cuadrantes (Alpha Cross)
    plt.axvline(0, color='gray', linestyle='--', alpha=0.6)
    plt.axhline(0, color='gray', linestyle='--', alpha=0.6)
    
    plt.title('Alpha Hunter: IA Sentiment vs Market Reality (6M)', fontsize=16, fontweight='bold')
    plt.xlabel('Sentiment Z-Score (IA)', fontsize=12)
    plt.ylabel('6-Month Price Return (%)', fontsize=12)
    
    # Labels Cuadrantes (Dinámico según rango)
    # Usamos max/min para posicionar texto en las esquinas
    xlims = plt.xlim()
    ylims = plt.ylim()
    
    # Top Right
    plt.text(xlims[1], ylims[1], 'WINNERS\n(Price Up + Sentiment Up)', ha='right', va='top', color='green', alpha=0.5)
    # Top Left
    plt.text(xlims[0], ylims[1], 'SKEPTICISM?\n(Price Up + Sentiment Down)', ha='left', va='top', color='orange', alpha=0.5)
    # Bottom Right
    plt.text(xlims[1], ylims[0], 'OPPORTUNITY?\n(Price Down + Sentiment Up)', ha='right', va='bottom', color='blue', alpha=0.5)
    # Bottom Left
    plt.text(xlims[0], ylims[0], 'LOSERS\n(Price Down + Sentiment Down)', ha='left', va='bottom', color='red', alpha=0.5)
    
    plt.grid(True, alpha=0.3)
    plt.show()
else:
    print("[!] No se obtuvieron resultados válidos para generar el gráfico.")